In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import Activation,BatchNormalization,Lambda,Input,Conv2D,Bidirectional,LSTM, Concatenate, MaxPooling2D,GlobalAveragePooling2D
import tensorflow.keras.backend as K
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from time import time
import datetime
import numpy as np
from generater import crnnGenerator
import logging

logger=tf.get_logger()
logger.setLevel(logging.ERROR)

In [2]:

def crnn():
    x=image_input=Input(name='image_input',shape=(256,32,1))
    """
    minx=Input(name='minx',shape=[1],dtype='float32')
    miny=Input(name='miny',shape=[1],dtype='float32')
    maxx=Input(name='maxx',shape=[1],dtype='float32')
    maxy=Input(name='maxy',shape=[1],dtype='float32')
    def cropimage(img,minx=0,miny=0,maxx=0,maxy=0):
        
        return tf.image.crop_to_bounding_box(img,minx,miny,maxx-minx,maxy-miny)
    x=Lambda(cropimage)([x,minx,miny,maxx,maxy])
    
    """
        
    x=Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='same', name='conv1_1')(x)#128 64 64
    x=MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='pool1', padding='same')(x)#64 32 64
    
    x=Conv2D(128, (3, 3), strides=(1, 1), activation='relu', padding='same', name='conv2_1')(x)#64 32 128
    x=MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='pool2', padding='same')(x)# 32 16 128

    x=Conv2D(256, (3, 3), strides=(1, 1), activation='relu', padding='same', name='conv3_1')(x)#32 16 256
    x=Conv2D(256, (3, 3), strides=(1, 1), activation='relu', padding='same', name='conv3_2')(x)#32 16 256
    x=MaxPooling2D(pool_size=(2, 2), strides=(1, 2), name='pool3', padding='same')(x)# 32 8 256
    
    x=Conv2D(512, (3, 3), strides=(1, 1), activation='relu', padding='same', name='conv4_1')(x)#32 8 512
    x=BatchNormalization(name='batchnorm1')(x)
    
    x=Conv2D(512, (3, 3), strides=(1, 1), activation='relu', padding='same', name='conv5_1')(x)#32 8 512
    
    x=BatchNormalization(name='batchnorm2')(x)
    x=MaxPooling2D(pool_size=(2, 2), strides=(1, 2),padding='valid', name='pool5')(x)# 32 4 512
    
    x=aa=Conv2D(512, (2, 2), strides=(1, 1), activation='relu', padding='valid', name='conv6_1')(x)# 32 4 512
    s=x.shape
    x=keras.layers.Reshape((s[1],s[3]),name='reshape_1')(x)#32 2048
    x=Bidirectional(LSTM(256, return_sequences=True,name='bidirectional_1'))(x)
    x=Bidirectional(LSTM(256, return_sequences=True,name='bidirectional_1'))(x)#32 512
    
    num_classes=87
    x=keras.layers.Dense(num_classes, name='dense1')(x)#알파벳+숫자#1953 36
    
    x=y_pred= Activation('softmax', name='softmax')(x)#1953 36
    
   # model_pred=keras.models.Model(inputs=[image_input,minx,miny,maxx,maxy],outputs=x)
    model_pred=keras.models.Model(inputs=image_input,outputs=x)
    
    maxstringlen=s[1]
    #maxstringlen=100
    
    def ctc_lambda_func(args):
        y_pred,labels, input_length, label_length = args
          #y_pred = y_pred[:, 2:, :] 
        return K.ctc_batch_cost(y_true=labels,y_pred=y_pred,input_length=input_length,label_length=label_length)    
        #return tf.nn.ctc_loss(labels=labels,logits=y_pred,logit_length=input_length,label_length=label_length,logits_time_major=False)    
    
    labels = Input(name='labels', shape=[maxstringlen], dtype='float32')
    input_length = Input(name='input_length', shape=[1], dtype='int64')
    label_length = Input(name='label_length', shape=[1], dtype='int64') 
    
    ctcloss=Lambda(ctc_lambda_func,output_shape=(1,),name='ctcloss')([y_pred,labels,input_length,label_length])
    
   # model_train=keras.models.Model(inputs=[image_input,minx,miny,maxx,maxy,labels,input_length,label_length],outputs=ctc_loss)
    model_train=keras.models.Model(inputs=[image_input,labels,input_length,label_length],outputs=ctcloss)
    
    return model_train,model_pred,maxstringlen

    

In [3]:
traincrnn,predcrnn,inputlength=crnn()
#predcrnn.save('crnnmodel.h5')

In [5]:
predcrnn.load_weights('outputmodel/crnnmodel/crnnweight.h5')

In [18]:
predcrnn.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     [(None, 256, 32, 1)]      0         
_________________________________________________________________
conv1_1 (Conv2D)             (None, 256, 32, 64)       640       
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 128, 16, 64)       0         
_________________________________________________________________
conv2_1 (Conv2D)             (None, 128, 16, 128)      73856     
_________________________________________________________________
pool2 (MaxPooling2D)         (None, 64, 8, 128)        0         
_________________________________________________________________
conv3_1 (Conv2D)             (None, 64, 8, 256)        295168    
_________________________________________________________________
conv3_2 (Conv2D)             (None, 64, 8, 256)        5900

In [7]:
import cv2

In [16]:
imgw=256
imgh=32

img=cv2.imread('D:/engocr/testmm/'+'sr01.jpg',cv2.IMREAD_GRAYSCALE)
img=cv2.resize(img,(imgw,imgh))

img=img.astype(np.float32)
img=(img / 255.0)
cv2.imshow('ada',img)
cv2.waitKey()
cv2.destroyAllWindows()

#imgs[0,:,:]=img
img=img.T
img=np.expand_dims(img,axis=-1)
img=np.expand_dims(img,axis=0)


In [17]:
b=predcrnn.predict(img)

In [18]:
res=[]
for data in b[0]:
    listdata=list(data)
    res.append( listdata.index(max(listdata)))
res
resu=[]
for i,abc in enumerate(res) :
    if abc!=86:
        resu.append(abc)
resu

[18, 17, 52, 53]

In [19]:
import string
def labels_to_text(labels):     
    alphabet = "abcdefghijklmnopqrstuvwxyz0123456789"
    alphabet87 = string.ascii_lowercase + string.ascii_uppercase + string.digits + ' +-*.,:!?%&$~/()[]<>"\'@#_'
    return ''.join(list(map(lambda x: alphabet87[int(x)], labels)))
labels_to_text(resu)

'sr01'

In [32]:
inputlength=np.ones((32,1))*(128//2)
inputlength

array([[64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.],
       [64.]])

In [13]:
def labels_to_text(labels):     
    alphabet = "abcdefghijklmnopqrstuvwxyz0123456789"
    return ''.join(list(map(lambda x: alphabet[int(x)], labels)))

In [15]:
labels_to_text(resu)

'm9'

In [9]:
b[0][0]

array([9.0169685e-04, 2.9561585e-01, 6.8642162e-02, 1.1974521e-01,
       8.0261542e-04, 4.7795489e-02, 2.9934574e-02, 7.2134426e-03,
       1.6024680e-04, 1.9210909e-02, 4.9673039e-03, 3.1123051e-02,
       1.5213985e-02, 9.1239428e-03, 2.8155730e-04, 3.1948622e-02,
       1.0874154e-03, 1.6102161e-02, 1.0888829e-01, 9.0564825e-03,
       3.7227737e-04, 1.2894981e-02, 1.0179144e-02, 4.9792108e-04,
       3.4431620e-03, 1.7110530e-03, 4.9436363e-05, 3.1823625e-03,
       2.4091445e-03, 5.9505890e-04, 2.0338944e-04, 9.7430765e-04,
       6.3139992e-04, 3.9370960e-04, 5.9121079e-04, 2.4338026e-04,
       1.4381313e-01], dtype=float32)

In [18]:
res=[]
for data in b[0]:
    listdata=list(data)
    res.append( listdata.index(max(listdata)))
res
resu=[]
for i,abc in enumerate(res) :
    if abc!=36:
        resu.append(abc)
resu

[14]

In [19]:
labels_to_text(resu)

'o'

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1